> 重新开始学习 另外一本书  Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow 2020-5-23 15:00:19，上一本书很多内容没有讲清楚。

- 总共有10章，一个周能学一章就不错。

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
tf.__version__

'2.1.0'

In [4]:
keras.__version__

'2.2.4-tf'

# introduction

## The perception
- TLU: step funcion after a linear regression 
- A Perceptron is simply composed of a single layer of TLUs
- serious weaknesses of Perceptrons—in particular, the fact that they are incapable of solving some trivial problems (e.g., the Exclusive OR (XOR) classification problem;
- MLP: Multilayer Perception

## The Multilayer Perceptron and Backpropagation

- For many years researchers struggled to find a way to train MLPs,
- backpropagation algorithm is able to compute the gradient of the network’s error with regard to every single model parameter.
- It is important to initialize all the hidden layers’ connection weights randomly
- activation function:
    - tanh: hyperbolic tangent/ which often helps speed up convergence/ (-1,1) 
    - RelU: rectified linear unit, defaults/ fast  

## Regression MLPs
- multivariate regression

## Classification MLPs
- cross-entropy loss (also called the log loss, see Chapter 4) is generally a good choice

## Implementing MLPs with Keras

In [5]:
fashion_mnist = keras.datasets.fashion_mnist

In [12]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 68s 15us/step


In [16]:
X_train_full.dtype

dtype('uint8')

In [15]:
type(X_train_full)

numpy.ndarray

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]